In [1]:
# import statements
from time import sleep
import json
from json import dumps
from kafka import KafkaProducer
import random
from datetime import datetime
import datetime as dt
import csv
import pandas as pd

In [2]:
terra_streaming = []
with open("FIT3182_Assignment/hotspot_TERRA_streaming.csv", 'r') as file:
    csvreader = csv.reader(file)
    header_terra_streaming = next(csvreader)
    for row in csvreader:
        terra_streaming.append(row)
        
terra_streaming = pd.DataFrame(terra_streaming)
terra_streaming.columns = header_terra_streaming

In [3]:
import pymongo
from pymongo import MongoClient

client = MongoClient ()
db = client.fit3182_assignment_db
merged_hist = db.merged_hist

date = (list(merged_hist.aggregate([
        { '$unwind' : '$Hotspot' },
        { '$sort' : { 'Hotspot.datetime' : -1}},
        { '$limit' : 1},
        { '$project' : {'Hotspot.datetime' : 1 , '_id' : 0}}
         ])))

temp = date[0]

l_date = temp['Hotspot']

latest_date = l_date['datetime']
latest_date = latest_date.replace("T"," ")
latest_date = latest_date.replace("-","/")
latest_date = datetime.strptime(latest_date,"%Y/%m/%d %H:%M:%S")
latest_date

datetime.datetime(2021, 12, 27, 4, 16, 51)

In [4]:
def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

In [5]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [6]:
if __name__ == '__main__':
   
    topic = 'Assignment'
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    row_no = terra_streaming.shape[0]
    latest_date = latest_date + dt.timedelta(days = 1)
    
    for e in range(20):
        latest_date = str(latest_date.strftime("%d/%m/%Y, %H:%M:%S"))
        row = random.randrange(0,(row_no - 1))
        terra_streaming.loc[row,"datetime"] = latest_date
        terra_streaming.loc[row,"key"] = 'Producer03'
        data = terra_streaming.loc[row,]
        data = data.to_json()
        publish_message(producer, topic, 'Producer03', data)
        latest_date = datetime.strptime(latest_date,"%d/%m/%Y, %H:%M:%S") + dt.timedelta(seconds = 2)
        sleep(1)

Publishing records..
Message published successfully. Data: {"latitude":"-36.3065","longitude":"141.4495","confidence":"69","surface_temperature_celcius":"44","datetime":"28\/12\/2021, 04:16:51","key":"Producer03"}
Message published successfully. Data: {"latitude":"-37.7741","longitude":"142.8188","confidence":"77","surface_temperature_celcius":"50","datetime":"28\/12\/2021, 04:16:53","key":"Producer03"}
Message published successfully. Data: {"latitude":"-36.3014","longitude":"143.203","confidence":"95","surface_temperature_celcius":"76","datetime":"28\/12\/2021, 04:16:55","key":"Producer03"}
Message published successfully. Data: {"latitude":"-38.0361","longitude":"146.0567","confidence":"100","surface_temperature_celcius":"90","datetime":"28\/12\/2021, 04:16:57","key":"Producer03"}
Message published successfully. Data: {"latitude":"-37.7211","longitude":"149.1342","confidence":"73","surface_temperature_celcius":"47","datetime":"28\/12\/2021, 04:16:59","key":"Producer03"}
Message publis